In [1]:
import pandas as pd
import numpy as np
import plotly

In [2]:
from wb_nlp.interfaces import elasticsearch

In [3]:
dr = elasticsearch.DocTopicAggregations(doc_class=elasticsearch.DocTopic, model_id="3e82ec784f125709c8bac46d7dd8a67f")

In [4]:
data = pd.DataFrame(dr.get_topic_profile_by_year_by_adm_region("topic_0"))

['Africa',
 'East Asia and Pacific',
 'Europe and Central Asia',
 'Latin America and Caribbean',
 'Middle East and North Africa',
 'South Asia',
 'The World Region']

In [33]:
year_field_topic_sum_df = data.pivot(index="year", columns="adm_region", values="adm_region_topic_sum").fillna(0)
year_field_topic_sum_df.head()

adm_region,Africa,East Asia and Pacific,Europe and Central Asia,Latin America and Caribbean,Middle East and North Africa,South Asia,The World Region
year,,,,,,,
1969-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1976-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1978-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1979-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1983-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
year_doc_count_df = data.pivot(index="year", columns="adm_region", values="year_doc_count").fillna(0)
year_doc_count_df.head()

adm_region,Africa,East Asia and Pacific,Europe and Central Asia,Latin America and Caribbean,Middle East and North Africa,South Asia,The World Region
year,,,,,,,
1969-01-01,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1976-01-01,6.0,0.0,0.0,6.0,6.0,6.0,0.0
1978-01-01,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1979-01-01,0.0,0.0,3.0,3.0,0.0,0.0,0.0
1983-01-01,2.0,2.0,0.0,0.0,0.0,0.0,0.0


In [50]:
data.drop_duplicates(subset=["year"])["year_doc_count"].sum()

98

In [41]:
year_topic_share_of_documents_df = (year_field_topic_sum_df / year_doc_count_df).fillna(0)

In [45]:
def get_data_payload(df):
    x = sorted(df.index)
    legend = sorted(df.columns)
    series = [dict(name=f, data=df.loc[x, f].tolist()) for f in legend]
    return dict(
        year=x,
        series=series,
        legend=legend,
    )

In [46]:
get_data_payload(year_topic_share_of_documents_df)

{'year': ['1969-01-01',
  '1976-01-01',
  '1978-01-01',
  '1979-01-01',
  '1983-01-01',
  '1985-01-01',
  '1986-01-01',
  '1987-01-01',
  '1988-01-01',
  '1990-01-01',
  '1991-01-01',
  '1993-01-01',
  '1994-01-01',
  '1996-01-01',
  '1997-01-01',
  '2000-01-01',
  '2001-01-01',
  '2002-01-01',
  '2003-01-01',
  '2004-01-01',
  '2005-01-01',
  '2006-01-01',
  '2007-01-01',
  '2008-01-01',
  '2009-01-01',
  '2010-01-01',
  '2011-01-01',
  '2012-01-01',
  '2013-01-01',
  '2014-01-01',
  '2015-01-01',
  '2016-01-01',
  '2017-01-01',
  '2018-01-01',
  '2019-01-01'],
 'series': [{'name': 'Africa',
   'data': [0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.12135424613952636,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0]},
  {'name': 'East Asia and Pacific',
   'data': [0.0,
    0.

In [23]:
yearly_field_topic_sum = data.pivot(index="year", columns="adm_region", values="adm_region_topic_sum").fillna(0).T.to_dict()
field_yearly_topic_sum = data.pivot(index="year", columns="adm_region", values="adm_region_topic_sum").fillna(0).to_dict()

In [31]:
import plotly.graph_objects as go

x = sorted(yearly_field_topic_sum)
fig = go.Figure()

for g in field_yearly_topic_sum:
    v = field_yearly_topic_sum[g]
    y = [v[i] for i in x]

    fig.add_trace(go.Scatter(
        x=x, y=y,
        mode='lines',
        line=dict(width=0.5),  # , color='rgb(184, 247, 212)'),
        stackgroup='teo',
        groupnorm='percent' # sets the normalization for the sum of the stackgroup
    ))


# fig.add_trace(go.Scatter(
#     x=x, y=[50, 70, 40, 60],
#     mode='lines',
#     line=dict(width=0.5, color='rgb(111, 231, 219)'),
#     stackgroup='one'
# ))
# fig.add_trace(go.Scatter(
#     x=x, y=[70, 80, 60, 70],
#     mode='lines',
#     line=dict(width=0.5, color='rgb(127, 166, 238)'),
#     stackgroup='one'
# ))
# fig.add_trace(go.Scatter(
#     x=x, y=[100, 100, 100, 100],
#     mode='lines',
#     line=dict(width=0.5, color='rgb(131, 90, 241)'),
#     stackgroup='one'
# ))

fig.update_layout(
    showlegend=True,
    xaxis_type='date',
    yaxis=dict(
        type='linear',
        range=[1, 100],
        ticksuffix='%'))

fig.show()

**PLEASE** save this file right now using the following naming convention: `NUMBER_FOR_SORTING-YOUR_INITIALS-SHORT_DESCRIPTION`, e.g. `1.0-fw-initial-data-exploration`. Use the number to order the file within the directory according to its usage.

In [3]:
## Hints

# nlp = spacy.load('en_core_web_sm')